### Experiment with MongoEngine

The ODM we use to access MongoDB

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document

from datetime import date

Open the database:

In [2]:
DB.open('pytest')

Make an account:

In [3]:
acct = Account(name='equity', category='Q')

Save it:

In [4]:
acct.save()

<Account: Account object>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [5]:
Account.objects

[<Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>]

In [6]:
Account.objects[0]

<Account: Account object>

In [7]:
acct = Account.objects[0]

In [8]:
acct.name

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [9]:
DB.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [10]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest')

In [11]:
db = DB.database

In [12]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [13]:
db.account.find_one()

{'_id': ObjectId('67dcc33dcaeb530a89508a83'),
 'name': 'equity',
 'category': 'Q'}

In [14]:
for c in db.list_collections():
    print(c)

{'name': 'transaction', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'M\x07\xe6o\xf2\x1bM\xb7\x9ek\xbfU\x0buk\x9f', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'entry', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'dM\x1a\x86A\rEG\x97k\xc1\x0bH#\xe6\x92', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'account', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'm\x99\xbf\x08V}M\xfb\xb1\xd6\x10\x83\xbf\x8a\xc5\x03', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}


In [15]:
for name in db.list_collection_names():
    print(name)

transaction
entry
account


In [16]:
db['account']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [17]:
db['account'].find_one()

{'_id': ObjectId('67dcc33dcaeb530a89508a83'),
 'name': 'equity',
 'category': 'Q'}

In [18]:
for obj in db['account'].find():
    print(obj)

{'_id': ObjectId('67dcc33dcaeb530a89508a83'), 'name': 'equity', 'category': 'Q'}
{'_id': ObjectId('67dcc33dcaeb530a89508a84'), 'name': 'yoyodyne', 'category': 'I'}
{'_id': ObjectId('67dcc33dcaeb530a89508a85'), 'name': 'checking', 'category': 'A'}
{'_id': ObjectId('67dcc33dcaeb530a89508a86'), 'name': 'amex', 'category': 'L'}
{'_id': ObjectId('67dcc33dcaeb530a89508a87'), 'name': 'visa', 'category': 'L'}
{'_id': ObjectId('67dcc33dcaeb530a89508a88'), 'name': 'groceries', 'category': 'E'}
{'_id': ObjectId('67dcc33dcaeb530a89508a89'), 'name': 'household', 'category': 'E'}
{'_id': ObjectId('67dcc33dcaeb530a89508a8a'), 'name': 'mortgage', 'category': 'E'}
{'_id': ObjectId('67dcc33dcaeb530a89508a8b'), 'name': 'car', 'category': 'E'}
{'_id': ObjectId('67dcc33dcaeb530a89508a8c'), 'name': 'travel', 'category': 'E'}
{'_id': ObjectId('67dcc34f450d53cfec4ab701'), 'name': 'equity', 'category': 'Q'}


### From Low Level to High Level

Question:  given a collection name ("account") can we find the corresponding MongoEngine class (Account)?

In [19]:
Document

mongoengine.document.Document

In [20]:
Document.__subclasses__()

[mongoengine.document.DynamicDocument,
 dexter.schema.Account,
 dexter.schema.Entry,
 dexter.schema.Transaction]

In [21]:
[cls for cls in Document.__subclasses__() if hasattr(cls, 'objects')]

[dexter.schema.Account, dexter.schema.Entry, dexter.schema.Transaction]

In [22]:
Account._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'account',
 'index_specs': []}

In [23]:
for cls in Document.__subclasses__():
    if not hasattr(cls, 'objects'):
        continue
    print(cls._meta['collection'], cls)

account <class 'dexter.schema.Account'>
entry <class 'dexter.schema.Entry'>
transaction <class 'dexter.schema.Transaction'>


### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [24]:
t = Transaction(description='hi', comment='aloha')

In [25]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [26]:
t.tags

[]

In [27]:
t.entries

[]

### Entries

In [28]:
p = Entry(uid='xxx', column='credit', date='2025-03-05', amount=1000, account='unknown')

In [29]:
p

<Entry: <En 2025-03-05 unknown -$1000.0>>

In [30]:
p.column

<Column.cr: 'credit'>

In [31]:
p.amount

1000.0

### References

The big test -- can we add that Entry to the transaction?

In [32]:
t.entries.append(p)

In [33]:
t.entries

[<Entry: <En 2025-03-05 unknown -$1000.0>>]

Yes!  🎉

### Misc Commands

In [34]:
db.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')

In [35]:
db.stats.find_one

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')>

In [36]:
db.list_collection_names()

['transaction', 'entry', 'account']

In [37]:
db.command('count','account')

{'n': 11, 'ok': 1.0}

In [38]:
db.command('hello')

{'isWritablePrimary': True,
 'topologyVersion': {'processId': ObjectId('67c4fbece8eb6b0fbfe3b917'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2025, 3, 21, 1, 39, 27, 618000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 814,
 'minWireVersion': 0,
 'maxWireVersion': 25,
 'readOnly': False,
 'ok': 1.0}

In [39]:
db.command('hostInfo')

{'system': {'currentTime': datetime.datetime(2025, 3, 21, 1, 39, 27, 623000),
  'hostname': 'cthulhu.local',
  'cpuAddrSize': 64,
  'memSizeMB': 65536,
  'memLimitMB': 65536,
  'numCores': 12,
  'numCoresAvailableToProcess': 12,
  'numPhysicalCores': 12,
  'numCpuSockets': 1,
  'cpuArch': 'arm64',
  'numaEnabled': False,
  'numNumaNodes': 1},
 'os': {'type': 'Darwin', 'name': 'Mac OS X', 'version': '24.3.0'},
 'extra': {'versionString': 'Darwin Kernel Version 24.3.0: Thu Jan  2 20:24:23 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T6020',
  'alwaysFullSync': 0,
  'nfsAsync': 0,
  'model': 'Mac14,13',
  'cpuString': 'Apple M2 Max',
  'pageSize': 16384,
  'scheduler': 'edge'},
 'ok': 1.0}

In [40]:
db.command('ping')

{'ok': 1.0}

### Fetch Transactions

Specify constraints on transactions

In [41]:
Transaction.objects

[<Transaction: <Tr 2024-01-02 yoyodyne -> checking $5000.0 paycheck>>, <Transaction: <Tr 2024-01-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-02-02 yoyodyne -> checking $5000.0 paycheck>>, <Transaction: <Tr 2024-02-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-01-05 checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-02-05 checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-01-10 visa -> car $50.0 Shell Oil>>, <Transaction: <Tr 2024-02-26 visa -> car $60.0 Shell Oil>>, <Transaction: <Tr 2024-01-04 checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-02-04 checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway

In [42]:
Transaction.objects(description='Safeway')

[<Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway>>]

In [43]:
for t in Transaction.objects(description='Safeway'):
    for e in t.entries:
        print(e.date, e.account, e.amount, e.column)

2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit


In [44]:
for t in Transaction.objects(description='Safeway'):
    print(t.accounts)

{'checking', 'groceries'}
{'visa', 'groceries'}
{'checking', 'groceries'}
{'visa', 'groceries'}


In [45]:
for t in Transaction.objects(description='Safeway'):
    print(t.pamount)

75.0
175.0
75.0
75.0


In [ ]:
for t in Transaction.objects(description='Safeway'):
    print(t.pdate, type(t.pdate))

In [ ]:
for t in Transaction.objects(description='Safeway'):
    print(t.originals)

In [ ]:
lst = list(Transaction.objects(description='Safeway'))

In [ ]:
lst[1].comment

In [ ]:
lst[1].pamount

In [ ]:
lst[1].pdate

In [ ]:
list(Transaction.objects(pamount__lt=175.0))

In [ ]:
for t in Transaction.objects(pdate=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

In [ ]:
for t in Transaction.objects(pdate__lte=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

### Operators

In [ ]:
for t in Transaction.objects(description__gte='Safeway'):
    print(t.pdate, t.description)

In [ ]:
for t in Transaction.objects(description__regex='^S'):
    print(t.pdate, t.description)

The operator automatically applies to list elements.

In [ ]:
for t in Transaction.objects(description__regex=r'\s'):
    print(t.pdate, t.description, t.pamount)

For compound constraints we need another class from MongoEngine.

In [ ]:
from mongoengine.queryset.visitor import Q

In [ ]:
for t in Transaction.objects(Q(description__regex=r'^S')):
    print(t.pdate, t.description)

In [ ]:
for t in Transaction.objects(Q(description__regex=r'^S') & Q(description__regex=r'\s')):
    print(t.pdate, t.description)

### QuerySet

In [ ]:
for a in Account.nominal_accounts:
    print(a.name)

### Combining Query Elements

In [ ]:
q = Q(description__regex=r'^S')

In [ ]:
q

In [ ]:
type(q)

In [ ]:
p = Q(description__regex=r'\s')

In [ ]:
p & q

In [ ]:
for t in Transaction.objects(p & q):
    print(t.pdate, t.description)

Create Q object using dictionaries

In [ ]:
dct = {'description__regex': r'^S'}

In [ ]:
Q(**dct)

Can an object have multiple constraints?

In [ ]:
dct = {'description__regex': r'^S', 'pamount__gt': 100}

In [ ]:
Q(**dct)

In [ ]:
for t in Transaction.objects(Q(**dct)):
    print(t.pdate, t.description, t.pamount)

Yep!

### Select Method

#### Select Transactions

All transactions:

In [ ]:
for t in DB.select(Transaction):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

By date:

In [ ]:
for t in DB.select(Transaction, date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, start_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, end_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

By amount:

In [ ]:
for t in DB.select(Transaction, amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
lst = DB.select(Transaction, amount=75)

In [ ]:
all(t.pamount == 75 for t in lst)

In [ ]:
for t in DB.select(Transaction, max_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, min_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

By description:

In [ ]:
for t in DB.select(Transaction, description = r'^s'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, comment=r'budget'):
    print(t.pdate, t.pamount, t.description, t.comment, t.pcredit, t.pdebit)

By account:

In [ ]:
for t in DB.select(Transaction, debit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, credit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

Some random combinations

In [ ]:
for t in DB.select(Transaction, description = r'^s', min_amount=100):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, start_date = date(2024,2,1), credit='visa'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

#### Select Entries

All entries:

In [ ]:
len(DB.select(Entry))

In [ ]:
for e in DB.select(Entry):
    print(e.date, e.account, e.amount, e.column)

By date:

In [ ]:
for e in DB.select(Entry, date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, start_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, end_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

By amount:

In [ ]:
for e in DB.select(Entry, amount=900):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, max_amount=900):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, min_amount=900):
    print(e.date, e.account, e.amount, e.column)

By account:

In [ ]:
for e in DB.select(Entry, account='groceries'):
    print(e.date, e.account, e.amount, e.column)

By column:

In [ ]:
for e in DB.select(Entry, column='credit'):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, column='debit'):
    print(e.date, e.account, e.amount, e.column)

### Serializing Objects

In [ ]:
import json
from bson.objectid import ObjectId
import datetime

In [ ]:
lst = DB.select(Transaction, start_date = date(2024,2,1), credit='visa')

In [ ]:
lst[0].to_json()

In [ ]:
type(lst[0])

In [ ]:
obj = Transaction.objects.as_pymongo()[0]

In [ ]:
type(obj)

In [ ]:
obj

In [ ]:
s = lst[0].to_json()

In [ ]:
json.loads(s)

In [ ]:
Transaction.from_json(s)

In [ ]:
s = 'account: {...:...}'

In [ ]:
s.find(':')

In [ ]:
s[:s.find(':')]

In [ ]:
s[s.find(':'):]